In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
import art
from dotenv import load_dotenv
import openai

load_dotenv()


api = art.UnslothAPI(wandb_project="agent-reinforcement-training")
model = await api._get_or_create_model(
    name="yes-or-no-unsloth-001",
    base_model="Qwen/Qwen2.5-7B-Instruct",
    _config={
        "init_args": {
            # "model_name": "<YOUR-MODEL-NAME>",
        },
    },
)

@art.retry(max_attempts=3)
async def rollout(client: openai.AsyncOpenAI, prompt: str) -> art.Trajectory:
    messages: art.Messages = [
        {
            "role": "user",
            "content": prompt,
        }
    ]
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name, max_tokens=100
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    if content == "yes":
        reward = 0.5
    elif content == "no":
        reward = 0.75
    elif content == "maybe":
        reward = 1.0
    else:
        reward = 0.0
    return art.Trajectory(messages_and_choices=[*messages, choice], reward=reward)


# Create a list of prompts for our model to respond to
prompts = []

# Define the different word combinations we want to test
word_combinations = [
    ["yes", "no", "maybe"],
    ["maybe", "yes", "no"],
    ["no", "yes", "maybe"],
    ["yes", "maybe", "no"],
    ["yes", "no"],
    ["maybe", "no"],
    ["no", "maybe"],
    ["no", "yes"],
    ["yes", "no"],
]

# Define the prefixes we want to use
prefixes = ["respond", "just respond"]

# For each combination of prefix, quote style, and words
for prefix in prefixes:
    for use_quotes in [True, False]:
        for words in word_combinations:
            # Format the prompt differently based on number of words
            if len(words) == 3:
                # For 3 words, join them with commas
                word_list = ", ".join([f'"{w}"' if use_quotes else w for w in words])
                prompt = f"{prefix} with {word_list}"
            else:
                # For 2 words, format as "word1 or word2"
                first_word = f'"{words[0]}"' if use_quotes else words[0]
                second_part = ""
                if len(words) > 1:
                    second_word = f'"{words[1]}"' if use_quotes else words[1]
                    second_part = f" or {second_word}"
                prompt = f"{prefix} with {first_word}{second_part}"
            
            # Add the formatted prompt to our list
            prompts.append(prompt)

openai_client = await model.openai_client()
for i in range(await model.get_iteration(), 1_000):
    train_groups = await art.gather_trajectories(
        ((rollout(openai_client, prompt) for _ in range(32)) for prompt in prompts),
        pbar_desc="train",
        stream_chat_completions=8,
        return_exceptions=False,
    )
    await model.tune(
        train_groups,
        config=art.TuneConfig(lr=1e-4, sequence_length=32768),
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.1. vLLM: 0.8.2.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.096 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 64.5%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.1 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32768. Num Sequences = 226.
Unsloth: vLLM's KV Cache can use up to 45.15 GB. Also swap space = 6 GB.
Unsloth: vLLM Bitsandbytes config using k

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.50s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.05it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.04s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.55s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.02it/s]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.07s/it]

Capturing CUDA graph shapes: 100%|██████████| 32/32 [00:29<00:00,  1.10it/s]
Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


train:   0%|          | 0/1152 [00:00<?, ?it/s]

NotFoundError: Error code: 404 - {'object': 'error', 'message': 'The model `yes-or-no-unsloth-001` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}